In [ ]:
import pandas as pd
import os
import json
import numpy as np
from itertools import groupby
import matplotlib.pyplot as plt
from scipy import stats,signal
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
import random
import re
import csv
from IPython.display import HTML, display, Image
import tabulate
import math as m
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import statsmodels.formula.api as smf
import itertools

mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['axes.linewidth'] = 1
#mpl.rcParams['xtick.major.size'] = 20
mpl.rcParams['xtick.major.width'] = 1
#mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['ytick.major.width'] = 1
mpl.rcParams['ytick.minor.width'] = 1
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

fday = [1,2,3,4,5,6,7,8,9,10]
fsession = [1,2]
unique_signals = [1,2,3,4]

In [ ]:
current_path = os.path.abspath(os.getcwd())
parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
grand_parent_path = os.path.abspath(os.path.join(parent_path, os.pardir))
main_path = os.path.abspath(os.path.join(grand_parent_path, os.pardir))

path_results = main_path+'/results/dots/'

In [ ]:
adf = pd.read_csv(path_results+'preanalyzed.csv')  
df_diff = pd.read_csv(path_results+'per_difficulty.csv')  
dfANOVA = pd.read_csv(path_results+'forANOVA.csv') 
dfLR = pd.read_csv(path_results+'linearRegr_diff.csv') 

adf = pd.merge(dfLR,adf,on='user_sessionID',how='outer') 
# sort df by userID and sessionID in second place
adf = adf.sort_values(by = ['userID', 'sessionID_y'])

userids = adf['userID'].unique()
userids = sorted(userids)
nsub = len(userids)
sessionids = adf['sessionID_x'].unique()
sessionids = sorted(sessionids)

# morning df
mdf = adf[adf['sessionID_x']%2==1]
# evening df
edf = adf[adf['sessionID_x']%2==0]
edf_sin_nan = edf.copy()
edf_sin_nan['sleep']=list(mdf['sleep'])

key_SR = ['mood','food','sleep', 'real_stress']
key_PV = ['Dsubj_optout_oo','Ssubj_optout_oo','subj_perf_no','OKubj_RT_no','DoverConf','SoverConf','risk_av']
key_LR = ['slope_RT', 'slope_DO', 'slope_SO', 'intercept_RT', 'intercept_DO','intercept_SO']
key_name = key_SR+key_PV

key_SR2plot = [key_SR[:int(len(key_SR)/2)],key_SR[int(len(key_SR)/2):]]
pair_keys_SR = list(itertools.combinations(key_SR, 2))

# pair of PV 
pair_keys_PV = list(itertools.combinations(key_PV[:-3], 2))
pair_keys_PV2plot = [pair_keys_PV[:int(len(pair_keys_PV)/2)],pair_keys_PV[int(len(pair_keys_PV)/2):]]
key_name_PV = {'Dsubj_optout_oo':'DO','Ssubj_optout_oo':'SO','subj_perf_no':'acc NO','OKubj_RT_no':'RT NO'}
pair_key_name_PV2plot = [[[key_name_PV[k] for k in tupl] for tupl in lista] for lista in pair_keys_PV2plot]

SR_color = {'mood': 'b','food': 'r','sleep': 'y', 'real_stress':'g'}

key_diff = ['Dperf_oo', 'Sperf_oo', 'perf_no', 'Dn_trials_oo', 'Sn_trials_oo', 'NOn_trials', 'DRT_oo', 'SRT_oo', \
            'DRT_OKoo', 'SRT_OKoo', 'RT_no', 'RT_noNOK', 'Doptout', 'Soptout']

In [ ]:
adf.head()

In [ ]:
adf.columns

In [ ]:
adf[['sessionID_x', 'userID','Dsubj_optout_oo','Ssubj_optout_oo','subj_perf_no','OKubj_RT_no','DoverConf','SoverConf','risk_av']].head()

In [ ]:
dfM = adf.groupby(['userID']).mean().reset_index()

In [ ]:
dfM.head()

In [ ]:
dfSD = adf.groupby(['userID']).std().reset_index()

In [ ]:
dfN = adf.groupby(['userID']).count().reset_index()

In [ ]:
plt.errorbar(dfM['userID'],dfM['Dsubj_optout_oo'], yerr=dfSD['Dsubj_optout_oo'], fmt="o",color=[1,0,0.4])
plt.errorbar(dfM['userID'],dfM['Ssubj_optout_oo'], yerr=dfSD['Ssubj_optout_oo'], fmt="o",color=[0.4,0,1])
plt.xlabel('participant number')
plt.xticks(ticks=dfM['userID'][::4], labels=np.arange(1,len(dfM['userID'])+1)[::4])
plt.ylabel('NT optout (%)')
plt.legend(['DO','SO'])
plt.axhline(0,color='k')
plt.tight_layout()
plt.savefig('raw_optout_per_subject.png')
plt.show()

In [ ]:
plt.errorbar(dfM['userID'],dfM['Ssubj_optout_oo'], yerr=dfSD['Ssubj_optout_oo'], fmt="o")
plt.xlabel('participant number')
plt.xticks(ticks=dfM['userID'][::4], labels=np.arange(len(dfM['userID']))[::4])
plt.ylabel('optout (%)')
plt.axhline(0,color='k')
plt.show()

In [ ]:
len(dfM)

In [ ]:
dfM['Dsubj_optout_oo'].mean()

In [ ]:
dfM['Dsubj_optout_oo'].std()

In [ ]:
dfM['Dsubj_optout_oo'].std()/np.sqrt(len(dfM['Dsubj_optout_oo']))

In [ ]:
dfM['Ssubj_optout_oo'].mean()

In [ ]:
dfM['Ssubj_optout_oo'].std()

In [ ]:
dfM['Ssubj_optout_oo'].std()/np.sqrt(len(dfM['Ssubj_optout_oo']))

In [ ]:
stats.ttest_rel(dfM['Dsubj_optout_oo'],dfM['Ssubj_optout_oo'])